In [24]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, fcluster
from backend.preprocess import preprocess_data
from sklearn.preprocessing import MinMaxScaler

df = preprocess_data("dataset/pcos_data.csv", train=True)
feature_names = [
    "Age (yrs)", "Weight (Kg)", "Height(Cm)", "BMI", "Reg.Exercise(Y/N)", 
    "Hair loss(Y/N)", "Pimples(Y/N)", "Skin darkening (Y/N)", "Fast food (Y/N)",
    "BMI*FastFood", "Exercise*HairLoss"
]
df = df[feature_names]
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)
pca = PCA(n_components=6)
df_pca = pca.fit_transform(df_scaled)
linkage_matrix = linkage(df_pca, method='ward')
df["Cluster_HC"] = fcluster(linkage_matrix, t=3, criterion='maxclust')
gmm = GaussianMixture(n_components=3, covariance_type='tied', random_state=42, n_init=50, weights_init=[1/3, 1/3, 1/3])
df["Cluster_GMM"] = gmm.fit_predict(df_pca)
with open("models/pcos_gmm_model.pkl", "wb") as file:
    pickle.dump((gmm, pca, feature_names), file)
print("Model training complete.")
print("Cluster Counts:", np.bincount(df["Cluster_GMM"]))
print("PCA Explained Variance:", pca.explained_variance_ratio_)


✅ Scaler trained and saved to d:\H&W\backend\../models/scaler.pkl
Model training complete.
Cluster Counts: [185 222 133]
PCA Explained Variance: [0.3469279  0.16673278 0.15001835 0.13455761 0.11014858 0.02956427]


In [17]:
import pickle

model_path = "models/pcos_gmm_model.pkl"

# Open the file and inspect its contents
with open(model_path, "rb") as file:
    loaded_data = pickle.load(file)

print("Contents of the model file:", type(loaded_data))
print("Number of elements in the file:", len(loaded_data) if isinstance(loaded_data, tuple) else "Not a tuple")

# Print each element if it's a tuple
if isinstance(loaded_data, tuple):
    for idx, item in enumerate(loaded_data):
        print(f"Element {idx+1}: Type={type(item)}")


Contents of the model file: <class 'tuple'>
Number of elements in the file: 3
Element 1: Type=<class 'sklearn.mixture._gaussian_mixture.GaussianMixture'>
Element 2: Type=<class 'sklearn.decomposition._pca.PCA'>
Element 3: Type=<class 'list'>


In [50]:
import os
print("Model exists:", os.path.exists("models/pcos_gmm_model.pkl"))


Model exists: True


In [9]:
import os
print(os.getcwd())  # ✅ This shows the current directory


d:\H&W


In [8]:
import os
print(os.chdir("d:/H&W/"))  # ✅ This changes the current directory

None
